In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from glob import glob

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

batchSize = 10000
## load images

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=batchSize, shuffle=True, **kwargs)
for batch_idx, (data, target) in enumerate(train_loader):
    images, target = data.to(device), target.to(device)
    break



#images = torch.rand((batchSize,1,28,28)).to(device)

## load model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net().to(device)
path = F"/content/gdrive/My Drive/project_321089724_302363965/MNIST Inception Score/mnist_cnn.pt"
filename = glob(path)[0]
model.load_state_dict(torch.load(filename))
preds = torch.exp(model(images)).detach().cpu().numpy()
print(np.mean(np.argmax(preds,axis=1)==target.cpu().numpy()))

cuda:0
0.9942


In [4]:
## calculate inception  score

splits=10 			# the number of splits to average the score over
classes = ["0","1","2","3","4","5","6","7","8","9"]
#print(preds)		# Predicted labels using the output of the transfer learnt architecture
scores = []
# Calculating the inception score
for i in range(splits):
    part = preds[np.argmax(preds,axis=1)==i]
    logp= np.log(part)
    self = np.sum(part*logp,axis=1)
    cross = np.mean(np.dot(part,np.transpose(logp)),axis=1)
    diff = self - cross
    kl = np.mean(self - cross)
    kl1 = []
    for j in range(splits):
        diffj = diff[(j * diff.shape[0] // splits):((j+ 1) * diff.shape[0] //splits)]
        kl1.append(np.exp(diffj.mean()))
    print("category: %s scores_mean = %.2f, scores_std = %.2f" % (classes[i], np.mean(kl1),np.std(kl1)))
    scores.append(np.exp(kl))
print("scores_mean = %.2f, scores_std = %.2f" % (np.mean(scores),
                                                 np.std(scores)))

category: 0 scores_mean = 1.13, scores_std = 0.12
category: 1 scores_mean = 1.05, scores_std = 0.03
category: 2 scores_mean = 1.23, scores_std = 0.08
category: 3 scores_mean = 1.16, scores_std = 0.11
category: 4 scores_mean = 1.13, scores_std = 0.08
category: 5 scores_mean = 1.20, scores_std = 0.09
category: 6 scores_mean = 1.09, scores_std = 0.06
category: 7 scores_mean = 1.18, scores_std = 0.13
category: 8 scores_mean = 1.20, scores_std = 0.11
category: 9 scores_mean = 1.18, scores_std = 0.07
scores_mean = 1.15, scores_std = 0.05


In [24]:
icp=[]
scores = []
splits = 10
for i in range(splits):
  part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
  kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
  kl = np.mean(np.sum(kl, 1))
  scores.append(np.exp(kl))

icp.append((np.mean(scores) , np.std(scores)))
print(scores)
print("Inception score is: %.4f, %.4f" % (np.mean(scores) , np.std(scores)))

[9.426944, 9.248535, 9.619477, 9.7021055, 9.41882, 9.547706, 9.467248, 9.531876, 9.371874, 9.609382]
Inception score is: 9.4944, 0.1276
